# 4o Lab: Παίζοντας Atari με βαθιά ενισχυτική μάθηση


# Αρχικοποιήσεις

## Tensorboard και ngrok

Αντικαταστήστε το authentication token σας που παίρνετε από το [ngrok](https://ngrok.com/) στη γραμμή που υποδεικνύεται. Όταν εκτελεστεί το κελί θα σας δώσει το URL όπου μπορείτε να βλέπετε το TensorBoard. Σημειώστε ότι σε περίπτωση επανεκκίνησης του πυρήνα θα πρέπει να ξανατρέξετε το κελί. Η διεύθυνση θα είναι διαφορετική, αλλά τα προηγούμενα στατιστικά σας δεν χάνονται (μέχρι να ανακυκλωθεί ο πυρήνας).

In [ ]:
!apt install apt-utils psmisc -y
!killall -9 ngrok
!rm -rf *ngrok*
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!mkdir -p ngrok-dir
!mv ngrok ngrok-dir/
# Στην επόμενη γραμμή βάλτε το δικό σας authentication token από το ngrok
!ngrok-dir/ngrok authtoken 1pcN37Yi5sx8yMipxMQcFDBRURT_2zt9dVjiaw8KXisyRNyE9

LOG_DIR = 'tb_log/'
!mkdir -p LOG_DIR

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('ngrok-dir/ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
print("Wait 10-15 seconds and then click the URL above to open TensorBoard")

## Εγκατάσταση βιβλιοθήκης και gym

In [2]:
!pip install --upgrade pip
!pip install --upgrade stable_baselines3[extra]
# we need a specific version of gym because of this issue: https://github.com/DLR-RM/stable-baselines3/issues/294
!pip install gym==0.17.3

     |████████████████████████████████| 1.5 MB 2.8 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.0
    Uninstalling pip-21.0:
      Successfully uninstalled pip-21.0
     |████████████████████████████████| 152 kB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 15.1 MB/s eta 0:00:01     |████████                        | 696 kB 15.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 2.8 MB/s eta 0:00:01


  Created wheel for gym: filename=gym-0.17.3-py3-none-any.whl size=1654653 sha256=e0a2d205abd1d0485499b6be574a6f5a7630d9879c9d74f28ad5a421d669a8aa
  Stored in directory: /root/.cache/pip/wheels/d1/81/4b/dd9c029691022cb957398d1f015e66b75e37637dda61abdf58
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.18.0
    Uninstalling gym-0.18.0:
      Successfully uninstalled gym-0.18.0


In [3]:
!pip install sb3-contrib

     |████████████████████████████████| 2.8 MB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 104 kB 12.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 10.4 MB/s eta 0:00:01
     |████████████████████████████████| 242 kB 17.1 MB/s eta 0:00:01


     |████████████████████████████████| 107 kB 19.5 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 1.6 MB/s  eta 0:00:01
     |████████████████████████████████| 90 kB 5.6 MB/s  eta 0:00:01
     |████████████████████████████████| 96 kB 3.5 MB/s  eta 0:00:01
     |████████████████████████████████| 84 kB 2.6 MB/s  eta 0:00:01
     |████████████████████████████████| 121 kB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 89 kB 559 kB/s eta 0:00:011
     |████████████████████████████████| 55 kB 2.5 MB/s  eta 0:00:01
  Created wheel for pytest-env: filename=pytest_env-0.6.2-py3-none-any.whl size=2370 sha256=fbebd4ae45c574e8beefef8e3cb378abaf027cbc3f13772c0cc62f28be0db56e
  Stored in directory: /root/.cache/pip/wheels/00/78/6f/8413b85149939cd491ef2c5d3c5422e49cb2f898c8402f81f7
  Created wheel for importlab: filename=importlab-0.6.1-py2.py3-none-any.whl size=21288 sha256=4794b05b43d2d3a70e3f2be1663e71e8b6253189b8098bd5e4234f95feee83f6
  Stored in directory: /root/.c

In [4]:
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
import datetime
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy
import base64
from pathlib import Path
from IPython import display as ipythondisplay
import numpy as np
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv
from stable_baselines3 import A2C
from sb3_contrib import QRDQN
from stable_baselines3 import PPO

## Καταγραφή video του actual gameplay του πράκτορα


In [ ]:
!apt-get install ffmpeg freeglut3-dev xvfb  -y # For visualization

In [ ]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
video_folder = '/kaggle/working/videos'

In [ ]:
def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [ ]:
def record_video(eval_env, model, video_length=500, prefix='', video_folder=video_folder):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  # Start the video at step=0 and record video_length steps
  env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = env.reset()
  for _ in range(video_length):
    #action, _ = model.predict(obs)
    #θα επιλέξουμε την ενέργεια με τις μεγαλύτερες πιθανότητες
    actions, _ = model.predict(obs, deterministic=True)
    obs, _, _, _ = env.step(actions)

  # Close the video recorder
  env.close()

## Random Agent

Αρχικά ορίζω ένα τυχαίο περιβάλλον για να τρέξει ο random agent

In [ ]:
atari_env_name='AsterixDeterministic-v4'


# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing. To DQN δεν επιτρέπει multi envs
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
# με ένα frame έχουμε τη θέση, με δύο την ταχύτητα, με τρία την επιτάχυνση και με τέσσερα τον ρυθμό μεταβολής της επιτάχυνσης (jerk)
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

Ορίζω ένα τυχαίο πράκτορα

In [ ]:
random_agent = DQN('CnnPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=1000)

Και τώρα τον τεστάρω:

In [ ]:
mean_reward, std_reward = evaluate_policy(random_agent, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

# Πειράματα Αρχικοποίησης Μοντέλου

Αρχικά ορίζω ένα περιβάλλον

In [ ]:
atari_env_name='AsterixDeterministic-v4'


# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing. To DQN δεν επιτρέπει multi envs
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
# με ένα frame έχουμε τη θέση, με δύο την ταχύτητα, με τρία την επιτάχυνση και με τέσσερα τον ρυθμό μεταβολής της επιτάχυνσης (jerk)
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

## Πειραμα CnnPolicy και 1000000 timesteps

In [ ]:
model_name='dqn-CnnpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

dqn_model = DQN('CnnPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=1000000
dqn_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(dqn_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
dqn_model.save("dqn_AsterixDeterministic-v4_CnnPolicy")

## Πειραμα CnnPolicy και 2000000 timesteps

In [ ]:
model_name='dqn-CnnpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

dqn_model = DQN('CnnPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=2000000
dqn_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(dqn_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
dqn_model.save("dqn_AsterixDeterministic-v4_CnnPolicy_big")

In [ ]:
mean_reward, std_reward = evaluate_policy( test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

## Πειραμα ΜlpPolicy και 1000000 timesteps

In [ ]:
model_name='dqn-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

dqn_model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=1000000
dqn_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(dqn_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
dqn_model.save("dqn_AsterixDeterministic-v4_version2")

## Πειραμα ΜlpPolicy και 2000000 timesteps

In [ ]:
model_name='dqn-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

dqn_model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=2000000
dqn_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(dqn_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
dqn_model.save("dqn_AsterixDeterministic-v4_version3")

## Deterministic-v4

### Πείραμα στο Deterministic-v4 με a2c

In [ ]:
atari_env_name='AsterixDeterministic-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing. To DQN δεν επιτρέπει multi envs
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
# με ένα frame έχουμε τη θέση, με δύο την ταχύτητα, με τρία την επιτάχυνση και με τέσσερα τον ρυθμό μεταβολής της επιτάχυνσης (jerk)
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

In [ ]:
model_name='a2c-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

a2c_model = A2C('CnnPolicy', env, verbose=1, tensorboard_log=model_log)

In [ ]:
max_steps=1000000
a2c_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(a2c_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
a2c_model.save("a2c_AsterixDeterministic-v4")

### Πείραμα στο Deterministic-v4 με QRDQN

In [ ]:
atari_env_name='AsterixDeterministic-v4'


# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing. To DQN δεν επιτρέπει multi envs
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
# με ένα frame έχουμε τη θέση, με δύο την ταχύτητα, με τρία την επιτάχυνση και με τέσσερα τον ρυθμό μεταβολής της επιτάχυνσης (jerk)
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

In [ ]:
model_name='QRDQN-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

QRDQN_model = QRDQN('CnnPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=1000000
QRDQN_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(QRDQN_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
QRDQN_model.save("QRDQN_AsterixDeterministic-v4")

### Πείραμα στο Deterministic-v4 με PPO

In [ ]:
model_name='PPO-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

PPO_model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=model_log)

In [ ]:
max_steps=1000000
PPO_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(PPO_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

# AsterixNoFrameskip-v4

### dqn

In [ ]:
atari_env_name='AsterixNoFrameskip-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing. To DQN δεν επιτρέπει multi envs
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
# με ένα frame έχουμε τη θέση, με δύο την ταχύτητα, με τρία την επιτάχυνση και με τέσσερα τον ρυθμό μεταβολής της επιτάχυνσης (jerk)
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

In [ ]:
model_name='dqn-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

dqn_model2 = DQN('CnnPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=1000000
dqn_model2.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(dqn_model2, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
dqn_model2.save("dqn_model_AsterixNoFrameskip-v4")

### a2c

In [ ]:
model_name='a2c-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

a2c_model2 = A2C('CnnPolicy', env, verbose=1, tensorboard_log=model_log)

In [ ]:
max_steps=1000000
a2c_model2.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(a2c_model2, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
a2c_model2.save("a2c_AsterixNoFrameskip-v4")

### QRDQN

In [ ]:
model_name='QRDQN-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

QRDQN_model = QRDQN('CnnPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=1000000
QRDQN_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(QRDQN_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
QRDQN_model.save("QRDQN_AsterixNoFrameskip-v4")

### PPO

In [ ]:
model_name='PPO-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

PPO_model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=model_log)

In [ ]:
max_steps=1000000
PPO_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(PPO_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

## Asterix-v4 

In [ ]:
atari_env_name='Asterix-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing. To DQN δεν επιτρέπει multi envs
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
# με ένα frame έχουμε τη θέση, με δύο την ταχύτητα, με τρία την επιτάχυνση και με τέσσερα τον ρυθμό μεταβολής της επιτάχυνσης (jerk)
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

### dqn

In [ ]:
model_name='dqn-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

dqn_model2 = DQN('CnnPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=1000000
dqn_model2.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(dqn_model2, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
dqn_model2.save("dqn_model_Asterix-v4")

### a2c

In [ ]:
model_name='a2c-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

a2c_model2 = A2C('CnnPolicy', env, verbose=1, tensorboard_log=model_log)

In [ ]:
max_steps=1000000
a2c_model2.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(a2c_model2, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
a2c_model2.save("a2c_model_Asterix-v4")

### QRDQN

In [ ]:
atari_env_name='Asterix-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing. To DQN δεν επιτρέπει multi envs
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
# με ένα frame έχουμε τη θέση, με δύο την ταχύτητα, με τρία την επιτάχυνση και με τέσσερα τον ρυθμό μεταβολής της επιτάχυνσης (jerk)
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

In [ ]:
model_name='QRDQN-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

QRDQN_model = QRDQN('CnnPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=1000000
QRDQN_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(QRDQN_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
QRDQN_model.save("QRDQN_Asterix-v4")

### PPO

In [ ]:
model_name='PPO-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

PPO_model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=model_log)

In [ ]:
max_steps=1000000
PPO_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(PPO_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

## Parametre Tunning

### gamma=0.95

In [ ]:
atari_env_name='Asterix-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing. To DQN δεν επιτρέπει multi envs
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
# με ένα frame έχουμε τη θέση, με δύο την ταχύτητα, με τρία την επιτάχυνση και με τέσσερα τον ρυθμό μεταβολής της επιτάχυνσης (jerk)
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

In [ ]:
model_name='dqn-TUN-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

dqn_model2 = DQN('CnnPolicy', env, verbose=1,gamma=0.95, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=1000000
dqn_model2.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(dqn_model2, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
dqn_model2.save("dqn_model_Asterix-v4_g_0.95")

QRDQN

In [ ]:
model_name='QRDQN-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

QRDQN_model = QRDQN('CnnPolicy', env, verbose=1, gamma=0.95, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=1000000
QRDQN_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(QRDQN_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

batch_size=16

In [ ]:
atari_env_name='Asterix-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing. To DQN δεν επιτρέπει multi envs
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
# με ένα frame έχουμε τη θέση, με δύο την ταχύτητα, με τρία την επιτάχυνση και με τέσσερα τον ρυθμό μεταβολής της επιτάχυνσης (jerk)
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

In [ ]:
model_name='dqn-Batch-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

dqn_model2 = DQN('CnnPolicy', env, verbose=1,gamma=0.95,batch_size=16, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=1000000
dqn_model2.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(dqn_model2, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

QRDQN testing batch size =16

In [ ]:
model_name='QRDQN-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

QRDQN_model = QRDQN('CnnPolicy', env, verbose=1, gamma=0.95, batch_size=16, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=1000000
QRDQN_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(QRDQN_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

### tau

In [ ]:
model_name='dqn-Batch-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

dqn_model2 = DQN('CnnPolicy', env, verbose=1,gamma=0.95, batch_size=16, tau=0.9, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=1000000
dqn_model2.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(dqn_model2, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
model_name='QRDQN-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

QRDQN_model = QRDQN('CnnPolicy', env, verbose=1, gamma=0.95, batch_size=16,tau=0.9, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=1000000
QRDQN_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(QRDQN_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

study learning rate

dqn

In [ ]:
atari_env_name='Asterix-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing. To DQN δεν επιτρέπει multi envs
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
# με ένα frame έχουμε τη θέση, με δύο την ταχύτητα, με τρία την επιτάχυνση και με τέσσερα τον ρυθμό μεταβολής της επιτάχυνσης (jerk)
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

In [ ]:
model_name='dqn-Batch-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

dqn_model2 = DQN('CnnPolicy', env, verbose=1,gamma=0.95, batch_size=16,learning_rate=0.00005, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=1000000
dqn_model2.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(dqn_model2, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
atari_env_name='Asterix-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing. To DQN δεν επιτρέπει multi envs
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
# με ένα frame έχουμε τη θέση, με δύο την ταχύτητα, με τρία την επιτάχυνση και με τέσσερα τον ρυθμό μεταβολής της επιτάχυνσης (jerk)
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

In [ ]:
model_name='QRDQN-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

QRDQN_model = QRDQN('CnnPolicy', env, verbose=1, gamma=0.95, batch_size=32,learning_rate=0.0005, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=1000000
QRDQN_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(QRDQN_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

## Τελικό Πείραμα

### Πρώτο

In [ ]:
atari_env_name='Asterix-v0'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing. To DQN δεν επιτρέπει multi envs
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
# με ένα frame έχουμε τη θέση, με δύο την ταχύτητα, με τρία την επιτάχυνση και με τέσσερα τον ρυθμό μεταβολής της επιτάχυνσης (jerk)
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

In [ ]:
model_name='dqn-Batch-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

dqn_model2 = DQN('CnnPolicy', env, verbose=1,gamma=0.95, batch_size=16, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=3000000
dqn_model2.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(dqn_model2, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
dqn_model2.save("final-v0")

In [ ]:
record_video(test_env,dqn_model2, video_length=5000, prefix='v0',video_folder = '/kaggle/working')

### Δεύτερο

In [ ]:
atari_env_name='Asterix-v4'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing. To DQN δεν επιτρέπει multi envs
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
# με ένα frame έχουμε τη θέση, με δύο την ταχύτητα, με τρία την επιτάχυνση και με τέσσερα τον ρυθμό μεταβολής της επιτάχυνσης (jerk)
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

In [ ]:
model_name='dqn-Batch-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

dqn_model2 = DQN('CnnPolicy', env, verbose=1,gamma=0.95, batch_size=16, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=3000000
dqn_model2.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(dqn_model2, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
dqn_model2.save("final-v4")

In [ ]:
record_video(test_env,dqn_model2, video_length=5000, prefix='v4',video_folder = '/kaggle/working')

Τρίτο Πείραμα

In [ ]:
atari_env_name='Asterix-v0'

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing. To DQN δεν επιτρέπει multi envs
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
# με ένα frame έχουμε τη θέση, με δύο την ταχύτητα, με τρία την επιτάχυνση και με τέσσερα τον ρυθμό μεταβολής της επιτάχυνσης (jerk)
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

In [ ]:
model_name='QRDQN-CnnPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

QRDQN_model = QRDQN('CnnPolicy', env, verbose=1, gamma=0.95, batch_size=32, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
max_steps=3000000
QRDQN_model.learn(total_timesteps=max_steps)

In [ ]:
mean_reward, std_reward = evaluate_policy(QRDQN_model, test_env, n_eval_episodes=10)
print(f"Eval reward: {mean_reward} (+/-{std_reward})")

In [ ]:
QRDQN_model.save("qrdqn-v0")

In [ ]:
record_video(test_env,QRDQN_model, video_length=5000, prefix='qrdqn-v0',video_folder = '/kaggle/working')

## Φόρτωση εκπαιδευμένου μοντέλου 


### Φόρτωση του final-v4

In [5]:
!wget --no-check-certificate 'https://www.dropbox.com/s/yvhx5qlvbohwurf/final-v4.zip?dl=1' -O final-v4.zip

--2021-03-16 01:50:39--  https://www.dropbox.com/s/yvhx5qlvbohwurf/final-v4.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/yvhx5qlvbohwurf/final-v4.zip [following]
--2021-03-16 01:50:39--  https://www.dropbox.com/s/dl/yvhx5qlvbohwurf/final-v4.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb37140f0b37e32c08e91887c86.dl.dropboxusercontent.com/cd/0/get/BKxo0I6x7sJRBc2ARjpzmNaIB_fj5O58uQchEtMys16J3GuTv5OYLCoLbI3doLT7nFRyuQ1eMSaXCRTdSB1ucMtUwY5S9i1npziOU-Vvi9rOCgIX3V4nWs7oxjCZyfgtGTDG97r_2KRv2hzaEUrsUhlY/file?dl=1# [following]
--2021-03-16 01:50:39--  https://ucb37140f0b37e32c08e91887c86.dl.dropboxusercontent.com/cd/0/get/BKxo0I6x7sJRBc2ARjpzmNaIB_fj5O58uQchEtMys16J3GuTv5OYLCoLbI3doLT7nFRyuQ1eMSaXCRTdSB1ucMtU

In [8]:
final_v4 = DQN.load("final-v4", verbose=1)

### Φόρτωση του final-v0

In [9]:
!wget --no-check-certificate 'https://www.dropbox.com/s/94nonly13wr2zjd/final-v0.zip?dl=1' -O final-v0.zip

--2021-03-16 01:54:37--  https://www.dropbox.com/s/94nonly13wr2zjd/final-v0.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/94nonly13wr2zjd/final-v0.zip [following]
--2021-03-16 01:54:37--  https://www.dropbox.com/s/dl/94nonly13wr2zjd/final-v0.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc90c9fd0be3b8d2f19896e1bdb2.dl.dropboxusercontent.com/cd/0/get/BKxJRyjVtyI1OuJ4JB4b_jh7pv_qQQJkU2V21HWdWr3Fk64Rr9oASkQs7MUgzdgsqrQNbL4b-MThiZ1yeB8p_K_YjpncVzA0LE5aQ4OSM3OTA8vBtrAANO70KGcAeuLx5PpptA7Ewt__-aAibvw9H2Q_/file?dl=1# [following]
--2021-03-16 01:54:38--  https://uc90c9fd0be3b8d2f19896e1bdb2.dl.dropboxusercontent.com/cd/0/get/BKxJRyjVtyI1OuJ4JB4b_jh7pv_qQQJkU2V21HWdWr3Fk64Rr9oASkQs7MUgzdgsqrQNbL4b-MThiZ1yeB8p_K_Y

In [10]:
final_v0 = DQN.load("final-v0", verbose=1)

### Φόρτωση του qrdqn-v0

In [ ]:
!wget --no-check-certificate 'https://www.dropbox.com/s/r33h3aiv5v1telf/qrdqn-v0.zip?dl=1' -O qrdqn-v0.zip

In [ ]:
qrdqn_v0 = QRDQN.load("qrdqn-v0", verbose=1)